# Machine Learning for NLP: Project 2

Lucas Artaud & Iswarya Sivasubramaniam DIA 1

In [102]:
import os
import pandas as pd
import torch
import sentencepiece
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from huggingface_hub import hf_hub_download
from transformers import pipeline
from gensim.models import KeyedVectors

In [2]:
torch.cuda.is_available()

True

## Data loading

In [3]:
folder = 'Traduction avis clients'

df = pd.DataFrame()

for i in range(1, 36):
    filename = f'avis_{i}_traduit.xlsx'
    path = os.path.join(folder, filename)
    df = pd.concat([df, pd.read_excel(path)], ignore_index=True)

df

,note,auteur,avis,assureur,produit,type,date_publication,date_exp,avis_en,avis_cor,avis_cor_en
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,train,06/09/2021,01/09/2021,"Best insurance, price, solutions, listening, s...",NaN,NaN
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,train,03/05/2021,01/05/2021,"I am generally satisfied, except that you have...",NaN,NaN
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,train,21/03/2021,01/03/2021,Very affordable price Several options are avai...,NaN,NaN
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,train,10/06/2021,01/06/2021,"I satisfy the service, a very fast response fr...",NaN,NaN
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,train,29/01/2017,01/01/2017,"Customer for more than 25 years, very disappoi...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
34430,NaN,mamybouly-129360,"Bonjour, j'ai une mauvaise expérience avec la ...",Sma,garantie-decennale,test,24/08/2021,01/08/2021,"Hello, I have a bad experience with the Socoba...","Bonjour, j'ai une mauvaise expérience avec la ...","Hello, I have a bad experience with the Socoba..."
34431,NaN,tarti-88637,Entreprise très impactée par le covid- secteur...,Allianz,assurances-professionnelles,test,18/05/2021,01/05/2021,Company very impacted by the covid- event sect...,Entreprise très impactée par le covid- secteur...,NaN
34432,NaN,victor-109599,"Suite a un cambriolage le 22/01/2021, je suis ...",Allianz,assurances-professionnelles,test,08/04/2021,01/04/2021,"Following a burglary on 01/22/2021, I am still...","Suite a un cambriolage le 22/01/2021, je suis ...",NaN
34433,NaN,lkio-110907,"Bonjour,\n\nJe constate qu'entre le moment de ...",Generali,assurances-professionnelles,test,19/04/2021,01/04/2021,"Hello,\n\nI note that between the time of my r...","Bonjour,\n\nJe constate qu'entre le moment de ...",NaN


## Data cleaning

In [4]:
df.drop(columns=['type', 'avis_en', 'avis_cor', 'avis_cor_en'], inplace=True)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017
...,...,...,...,...,...,...,...
34430,NaN,mamybouly-129360,"Bonjour, j'ai une mauvaise expérience avec la ...",Sma,garantie-decennale,24/08/2021,01/08/2021
34431,NaN,tarti-88637,Entreprise très impactée par le covid- secteur...,Allianz,assurances-professionnelles,18/05/2021,01/05/2021
34432,NaN,victor-109599,"Suite a un cambriolage le 22/01/2021, je suis ...",Allianz,assurances-professionnelles,08/04/2021,01/04/2021
34433,NaN,lkio-110907,"Bonjour,\n\nJe constate qu'entre le moment de ...",Generali,assurances-professionnelles,19/04/2021,01/04/2021


In [5]:
df['avis'] = df['avis'].replace('\n', ' ', regex=True)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017
...,...,...,...,...,...,...,...
34430,NaN,mamybouly-129360,"Bonjour, j'ai une mauvaise expérience avec la ...",Sma,garantie-decennale,24/08/2021,01/08/2021
34431,NaN,tarti-88637,Entreprise très impactée par le covid- secteur...,Allianz,assurances-professionnelles,18/05/2021,01/05/2021
34432,NaN,victor-109599,"Suite a un cambriolage le 22/01/2021, je suis ...",Allianz,assurances-professionnelles,08/04/2021,01/04/2021
34433,NaN,lkio-110907,"Bonjour, Je constate qu'entre le moment de ma...",Generali,assurances-professionnelles,19/04/2021,01/04/2021


In [6]:
df.isna().sum()

note                10331
auteur                  1
avis                    0
assureur                0
produit                 0
date_publication        0
date_exp                0
dtype: int64

In [7]:
df.dropna(inplace=True)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017
...,...,...,...,...,...,...,...
24099,1.0,hophop-107522,Assurance moto chez la mutuel des motards en F...,Mutuelle des Motards,moto,22/03/2021,01/03/2021
24100,1.0,tzl-81680,Même les demandes les plus simples n'aboutisse...,Allianz,habitation,06/12/2019,01/12/2019
24101,1.0,jmr-72500-110395,"En décembre 2019, j'ai souscrit à un contrat C...",Cegema Assurances,sante,14/04/2021,01/04/2021
24102,3.0,cris-77532,Je suis assurer à la gmf depuis plus de 15 ans...,GMF,auto,11/07/2019,01/07/2019


In [8]:
df_subset = df.head(10)

## Summary and translation

### Summary

In [9]:
summarizer_pipeline = pipeline("summarization", min_length=10, max_length=30)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
def summarizer(text):
    result = summarizer_pipeline(text)
    return result[0]['summary_text']

In [11]:
df_subset['résumé'] = df_subset['avis'].apply(summarizer)
df_subset

C:\Users\lucas\AppData\Local\Temp\ipykernel_21848\4062133327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['résumé'] = df_subset['avis'].apply(summarizer)


,note,auteur,avis,assureur,produit,date_publication,date_exp,résumé
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021,"Meilleurs assurances, prix, solutions, écoute..."
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021,"je suis globalement satisfait , sauf que vous..."
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021,Prix tres abordable plusieurs options s'offre...
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021,"Cordialement. je satisfait du service, une ré..."
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017,"Client depuis plus de 25 years, très déçu de ..."
5,1.0,spam-62064,"Vente forcée par téléphone, prétextant des évè...",Néoliane Santé,sante,06/03/2018,01/03/2018,Néoliane va dire que c'est la faute de la pla...
6,3.0,ludovic-b-135966,Nouveau client satisfait du prix et des servic...,APRIL,sante,05/10/2021,01/10/2021,Nouveau client satisfait du prix et des servi...
7,1.0,laetitia741-97523,Un assureur m'a contacté par telephone pour me...,SantéVet,animaux,17/09/2020,01/09/2020,Un assureur m'a contacté par telephone pour m...
8,1.0,gbernard10-102485,Je ne peux que rejoindre les nombreux avis nég...,Mercer,sante,12/01/2021,01/01/2021,"""Nous avons du retard mais vos demandes vont ..."
9,1.0,rezo31-62544,GENERALI ASSURANCES VIE PATRIMOINE Très mauvai...,Generali,vie,21/03/2018,01/03/2018,Generali ASSURANCES VIE PATRIMOINE . Très mau...


### Translation

In [12]:
fr_en_translator_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

c:\Users\lucas\Desktop\GitHub\NLP_project2\NLP_venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
def fr_en_translator(text):
    result = fr_en_translator_pipeline(text)
    return result[0]['translation_text']

In [14]:
df_subset['traduction'] = df_subset['avis'].apply(fr_en_translator)
df_subset

C:\Users\lucas\AppData\Local\Temp\ipykernel_21848\3255095628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['traduction'] = df_subset['avis'].apply(fr_en_translator)


,note,auteur,avis,assureur,produit,date_publication,date_exp,résumé,traduction
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021,"Meilleurs assurances, prix, solutions, écoute...","Best insurance, prices, solutions, listening, ..."
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021,"je suis globalement satisfait , sauf que vous...","I'm generally satisfied, except that you have ..."
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021,Prix tres abordable plusieurs options s'offre...,Very affordable prices several options are ava...
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021,"Cordialement. je satisfait du service, une ré...","I am satisfied with the service, a very quick ..."
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017,"Client depuis plus de 25 years, très déçu de ...","Customer for more than 25 years, very disappoi..."
5,1.0,spam-62064,"Vente forcée par téléphone, prétextant des évè...",Néoliane Santé,sante,06/03/2018,01/03/2018,Néoliane va dire que c'est la faute de la pla...,"Forcible sale by telephone, implying serious e..."
6,3.0,ludovic-b-135966,Nouveau client satisfait du prix et des servic...,APRIL,sante,05/10/2021,01/10/2021,Nouveau client satisfait du prix et des servi...,New customer satisfied with the price and the ...
7,1.0,laetitia741-97523,Un assureur m'a contacté par telephone pour me...,SantéVet,animaux,17/09/2020,01/09/2020,Un assureur m'a contacté par telephone pour m...,An insurer contacted me by telephone to sell m...
8,1.0,gbernard10-102485,Je ne peux que rejoindre les nombreux avis nég...,Mercer,sante,12/01/2021,01/01/2021,"""Nous avons du retard mais vos demandes vont ...",I can only join the many negative reviews. I e...
9,1.0,rezo31-62544,GENERALI ASSURANCES VIE PATRIMOINE Très mauvai...,Generali,vie,21/03/2018,01/03/2018,Generali ASSURANCES VIE PATRIMOINE . Très mau...,GENERALI INSURANCE LIFE HERITAGE Very bad cust...


## Sentiment analysis

### Predict positive or negative

In [15]:
predict_pos_neg_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [16]:
def predict_pos_neg(text):
    result = predict_pos_neg_pipeline(text)
    return result[0]['label']

In [17]:
df_subset['pos_neg_pred'] = df_subset['avis'].apply(predict_pos_neg)
df_subset

C:\Users\lucas\AppData\Local\Temp\ipykernel_21848\2808179175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['pos_neg_pred'] = df_subset['avis'].apply(predict_pos_neg)


,note,auteur,avis,assureur,produit,date_publication,date_exp,résumé,traduction,pos_neg_pred
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021,"Meilleurs assurances, prix, solutions, écoute...","Best insurance, prices, solutions, listening, ...",POSITIVE
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021,"je suis globalement satisfait , sauf que vous...","I'm generally satisfied, except that you have ...",NEGATIVE
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021,Prix tres abordable plusieurs options s'offre...,Very affordable prices several options are ava...,NEGATIVE
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021,"Cordialement. je satisfait du service, une ré...","I am satisfied with the service, a very quick ...",POSITIVE
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017,"Client depuis plus de 25 years, très déçu de ...","Customer for more than 25 years, very disappoi...",NEGATIVE
5,1.0,spam-62064,"Vente forcée par téléphone, prétextant des évè...",Néoliane Santé,sante,06/03/2018,01/03/2018,Néoliane va dire que c'est la faute de la pla...,"Forcible sale by telephone, implying serious e...",NEGATIVE
6,3.0,ludovic-b-135966,Nouveau client satisfait du prix et des servic...,APRIL,sante,05/10/2021,01/10/2021,Nouveau client satisfait du prix et des servi...,New customer satisfied with the price and the ...,POSITIVE
7,1.0,laetitia741-97523,Un assureur m'a contacté par telephone pour me...,SantéVet,animaux,17/09/2020,01/09/2020,Un assureur m'a contacté par telephone pour m...,An insurer contacted me by telephone to sell m...,NEGATIVE
8,1.0,gbernard10-102485,Je ne peux que rejoindre les nombreux avis nég...,Mercer,sante,12/01/2021,01/01/2021,"""Nous avons du retard mais vos demandes vont ...",I can only join the many negative reviews. I e...,NEGATIVE
9,1.0,rezo31-62544,GENERALI ASSURANCES VIE PATRIMOINE Très mauvai...,Generali,vie,21/03/2018,01/03/2018,Generali ASSURANCES VIE PATRIMOINE . Très mau...,GENERALI INSURANCE LIFE HERITAGE Very bad cust...,NEGATIVE


### Predict star

In [18]:
predict_stars_pipeline = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

In [19]:
def predict_stars(text):
    result = predict_stars_pipeline(text[:512])
    return result[0]['label'][0]

In [20]:
df_subset['stars_pred'] = df_subset['avis'].apply(predict_stars)
df_subset

C:\Users\lucas\AppData\Local\Temp\ipykernel_21848\4274888546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['stars_pred'] = df_subset['avis'].apply(predict_stars)


,note,auteur,avis,assureur,produit,date_publication,date_exp,résumé,traduction,pos_neg_pred,stars_pred
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021,"Meilleurs assurances, prix, solutions, écoute...","Best insurance, prices, solutions, listening, ...",POSITIVE,5
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021,"je suis globalement satisfait , sauf que vous...","I'm generally satisfied, except that you have ...",NEGATIVE,4
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021,Prix tres abordable plusieurs options s'offre...,Very affordable prices several options are ava...,NEGATIVE,4
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021,"Cordialement. je satisfait du service, une ré...","I am satisfied with the service, a very quick ...",POSITIVE,5
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017,"Client depuis plus de 25 years, très déçu de ...","Customer for more than 25 years, very disappoi...",NEGATIVE,1
5,1.0,spam-62064,"Vente forcée par téléphone, prétextant des évè...",Néoliane Santé,sante,06/03/2018,01/03/2018,Néoliane va dire que c'est la faute de la pla...,"Forcible sale by telephone, implying serious e...",NEGATIVE,1
6,3.0,ludovic-b-135966,Nouveau client satisfait du prix et des servic...,APRIL,sante,05/10/2021,01/10/2021,Nouveau client satisfait du prix et des servi...,New customer satisfied with the price and the ...,POSITIVE,4
7,1.0,laetitia741-97523,Un assureur m'a contacté par telephone pour me...,SantéVet,animaux,17/09/2020,01/09/2020,Un assureur m'a contacté par telephone pour m...,An insurer contacted me by telephone to sell m...,NEGATIVE,1
8,1.0,gbernard10-102485,Je ne peux que rejoindre les nombreux avis nég...,Mercer,sante,12/01/2021,01/01/2021,"""Nous avons du retard mais vos demandes vont ...",I can only join the many negative reviews. I e...,NEGATIVE,1
9,1.0,rezo31-62544,GENERALI ASSURANCES VIE PATRIMOINE Très mauvai...,Generali,vie,21/03/2018,01/03/2018,Generali ASSURANCES VIE PATRIMOINE . Très mau...,GENERALI INSURANCE LIFE HERITAGE Very bad cust...,NEGATIVE,1


## Word2Vec

### Vectorise reviews

In [21]:
word2vec_model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/wikipedia2vec_frwiki_20180420_300d", filename="frwiki_20180420_300d.txt"))

In [22]:
def vectorize_sentence(sentence):
    words = sentence.split()  # Supposons que vos avis soient tokenisés par des espaces
    vectors = [word2vec_model[word] for word in words if word in word2vec_model.key_to_index]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

In [23]:
df['avis_vectorisé'] = df['avis'].apply(vectorize_sentence)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp,avis_vectorisé
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021,"[0.24681999, 0.16044001, 0.06285334, -0.208206..."
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021,"[0.23545946, 0.0059973006, 0.07420811, -0.1272..."
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021,"[0.2317636, 0.08315454, 0.08915909, -0.1495227..."
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021,"[0.20486194, -0.07464761, 0.013419046, -0.1545..."
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017,"[0.25692186, 0.04158, 0.057759054, -0.12208381..."
...,...,...,...,...,...,...,...,...
24099,1.0,hophop-107522,Assurance moto chez la mutuel des motards en F...,Mutuelle des Motards,moto,22/03/2021,01/03/2021,"[0.2766443, 0.0641923, 0.1040519, -0.056559607..."
24100,1.0,tzl-81680,Même les demandes les plus simples n'aboutisse...,Allianz,habitation,06/12/2019,01/12/2019,"[0.20751432, 0.07767143, 0.0675238, -0.0785952..."
24101,1.0,jmr-72500-110395,"En décembre 2019, j'ai souscrit à un contrat C...",Cegema Assurances,sante,14/04/2021,01/04/2021,"[0.24621597, 0.058450233, 0.08599951, -0.08418..."
24102,3.0,cris-77532,Je suis assurer à la gmf depuis plus de 15 ans...,GMF,auto,11/07/2019,01/07/2019,"[0.2087177, 0.039839808, 0.082023, -0.07003894..."


In [24]:
len(df.iloc[8]['avis_vectorisé'])

300

### Word2Vec Training

In [43]:
df.dropna(inplace=True)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp,avis_vectorisé
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021,"[0.24681999, 0.16044001, 0.06285334, -0.208206..."
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021,"[0.23545946, 0.0059973006, 0.07420811, -0.1272..."
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021,"[0.2317636, 0.08315454, 0.08915909, -0.1495227..."
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021,"[0.20486194, -0.07464761, 0.013419046, -0.1545..."
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017,"[0.25692186, 0.04158, 0.057759054, -0.12208381..."
...,...,...,...,...,...,...,...,...
24099,1.0,hophop-107522,Assurance moto chez la mutuel des motards en F...,Mutuelle des Motards,moto,22/03/2021,01/03/2021,"[0.2766443, 0.0641923, 0.1040519, -0.056559607..."
24100,1.0,tzl-81680,Même les demandes les plus simples n'aboutisse...,Allianz,habitation,06/12/2019,01/12/2019,"[0.20751432, 0.07767143, 0.0675238, -0.0785952..."
24101,1.0,jmr-72500-110395,"En décembre 2019, j'ai souscrit à un contrat C...",Cegema Assurances,sante,14/04/2021,01/04/2021,"[0.24621597, 0.058450233, 0.08599951, -0.08418..."
24102,3.0,cris-77532,Je suis assurer à la gmf depuis plus de 15 ans...,GMF,auto,11/07/2019,01/07/2019,"[0.2087177, 0.039839808, 0.082023, -0.07003894..."


In [116]:
X_train, X_test, y_train, y_test = train_test_split(df['avis_vectorisé'], df['note'], test_size=0.2, random_state=42)

In [117]:
X_train

15626    [0.31174093, 0.14216363, 0.10077727, -0.093154...
8878     [0.24600479, 0.08721428, 0.09521428, -0.109495...
16667    [0.19319166, 0.10429168, 0.051700007, -0.08747...
17013    [0.2423368, 0.026278943, 0.06603422, -0.068026...
5249     [0.22231737, 0.007386956, 0.13346955, -0.11153...
                               ...                        
21831    [0.24887681, 0.008658931, 0.035766065, -0.1146...
5455     [0.28149566, 0.087247826, 0.029582608, -0.0353...
870      [0.25010556, 0.06342223, 0.11823889, -0.080649...
15978    [0.19935709, 0.05392856, 0.07910613, -0.061081...
23934    [0.19332175, 0.12376522, 0.12292175, -0.072447...
Name: avis_vectorisé, Length: 19058, dtype: object

In [118]:
y_train

15626    3.0
8878     5.0
16667    3.0
17013    1.0
5249     3.0
        ... 
21831    1.0
5455     4.0
870      4.0
15978    4.0
23934    3.0
Name: note, Length: 19058, dtype: float64

In [129]:
# model = LogisticRegression()
model = LinearRegression()
model.fit(X_train.tolist(), y_train)

LinearRegression()

In [130]:
y_pred = model.predict(X_test.tolist())
y_pred

array([1.5397983, 2.389738 , 2.470921 , ..., 3.3712535, 1.9956107,
       1.2819839], dtype=float32)

In [131]:
y_pred = np.round(y_pred).astype(int)
y_pred

array([2, 2, 2, ..., 3, 2, 1])

In [132]:
print('RMSE =', mean_squared_error(y_test, y_pred, squared=False))

RMSE = 1.0894899988079851


In [133]:
print('R² =', r2_score(y_test, y_pred))

R² = 0.49437446746814795
